In [131]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [132]:
# Variables

TOTAL_IMAGES = 420 # total unique: 2160 / total unique where trait count is a trait: 12960

all_images = []

background = ["blue", "gold", "green", "peach", "pink"]
color = ["black", "blue", "deepblue", "green", "orange", "purple", "red", "white", "yellow"]
spots = ["black", "ice", "white"]
floor = ["grass", "mud", "sand", "snow"]
toxicity = ["safe", "toxic"]
species = ["toadstool", "magic"]

# add magic mushroom, number of mushrooms, MAYBE add weather/landscape instead of floor?

background_weights = [10, 5, 15, 40, 30]
color_weights = [1, 2, 5, 15, 25, 11, 32, 1, 8]
spots_weights = [20, 5, 75]
floor_weights = [50, 30, 15, 5]
toxicity_weights = [85, 15]
species_weights = [95, 5]


In [133]:
def create_new_image():
    new_image = {}

    # For each trait category, select a random trait based on the weightings
    new_image["Background"] = random.choices(background, background_weights)[0]
    new_image["Color"] = random.choices(color, color_weights)[0]
    new_image["Spots"] = random.choices(spots, spots_weights)[0]
    new_image["Floor"] = random.choices(floor, floor_weights)[0]
    new_image["Toxicity"] = random.choices(toxicity, toxicity_weights)[0]
    new_image["Species"] = random.choices(species, species_weights)[0]

    if new_image in all_images:
        return create_new_image()
    else:
        return new_image


for i in range(TOTAL_IMAGES):
    new_image = create_new_image()

    all_images.append(new_image)


In [134]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [135]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [136]:
# print(all_images)

In [137]:
# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0

color_count = {}
for item in color:
    color_count[item] = 0

spots_count = {}
for item in spots:
    spots_count[item] = 0

floor_count = {}
for item in floor:
    floor_count[item] = 0

toxicity_count = {}
for item in toxicity:
    toxicity_count[item] = 0

species_count = {}
for item in species:
    species_count[item] = 0

for image in all_images:
    background_count[image["Background"]] += 1
    color_count[image["Color"]] += 1
    spots_count[image["Spots"]] += 1
    floor_count[image["Floor"]] += 1
    toxicity_count[image["Toxicity"]] += 1
    species_count[image["Species"]] += 1

print(background_count)
print(color_count)
print(spots_count)
print(floor_count)
print(toxicity_count)
print(species_count)


{'blue': 66, 'gold': 48, 'green': 79, 'peach': 124, 'pink': 103}
{'black': 11, 'blue': 18, 'deepblue': 33, 'green': 66, 'orange': 80, 'purple': 53, 'red': 94, 'white': 18, 'yellow': 47}
{'black': 132, 'ice': 44, 'white': 244}
{'grass': 162, 'mud': 130, 'sand': 91, 'snow': 37}
{'safe': 296, 'toxic': 124}
{'toadstool': 354, 'magic': 66}


In [138]:
# Generate Metadata for all Traits
METADATA_FILE_NAME = './metadata/all-traits.json'
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

print("Created all-traits JSON file")


Created all-traits JSON file


In [139]:
# Generate Images
for item in all_images:

    # print("sourcing images for", item)

    # source all images
    # BASE IMAGES
    bgImage = Image.open(
        f'./trait-layers/backgrounds/{item["Background"]}.png').convert('RGBA')
    toadstoolColor = Image.open(
        f'./trait-layers/colors/{item["Color"]}-toadstool.png').convert('RGBA')
    toadstoolOutline = Image.open(
        f'./trait-layers/general/outline-toadstool.png').convert('RGBA')
    floorImage = Image.open(
        f'./trait-layers/floors/{item["Floor"]}.png').convert('RGBA')

    # CONDITIONALLY IMPORT IMAGES    
    if item["Species"] == "magic":
        magicColor = Image.open(
            f'./trait-layers/colors/{item["Color"]}-magic.png').convert('RGBA')
        magicOutline = Image.open(
            f'./trait-layers/general/outline-magic.png').convert('RGBA')
        shadingImage = Image.open(
            f'./trait-layers/general/shading-magic.png').convert('RGBA')
    else:    
        shadingImage = Image.open(
            f'./trait-layers/general/shading-toadstool.png').convert('RGBA')
        toadstoolSpots = Image.open(
            f'./trait-layers/spots/{item["Spots"]}.png').convert('RGBA')

    # properties to change only when item is of toxic variety
    if item["Toxicity"] == "toxic":
        toxicImage = Image.open(
            f'./trait-layers/general/toxic.png').convert('RGBA')
        # change the alpha channels of the images for giving the toxic-look
        toxicImage.putalpha(50)
        shadingImage.putalpha(144)

    # layer the images together
    tsColWithBg = Image.alpha_composite(bgImage, toadstoolColor)
    toadstoolBaseImage = Image.alpha_composite(tsColWithBg, toadstoolOutline)

    # create the magic base (outline & color) if required
    if item["Species"] == "magic":
        magicBaseImage = Image.alpha_composite(magicColor, magicOutline)
        preshadedImage = Image.alpha_composite(toadstoolBaseImage, magicBaseImage)
    else:
        preshadedImage = Image.alpha_composite(toadstoolBaseImage, toadstoolSpots)

    # add the shading
    shadedImage = Image.alpha_composite(preshadedImage, shadingImage)

    # add the floor
    finalImage = Image.alpha_composite(shadedImage, floorImage)

    # add toxic odour lines (if toxic)
    if item["Toxicity"] == "toxic":
        finalImage = Image.alpha_composite(finalImage, toxicImage)

    # Convert to RGB
    rgb_im = finalImage.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)


In [140]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"].title()))
    token["attributes"].append(getAttribute("Color", i["Color"].title()))
    token["attributes"].append(getAttribute("Spots", i["Spots"].title()))
    token["attributes"].append(getAttribute("Floor", i["Floor"].title()))
    token["attributes"].append(getAttribute("Toxicity", i["Toxicity"].title()))
    token["attributes"].append(getAttribute("Species", i["Species"].title()))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()